In [2]:
import pandas as pd
import numpy as np
import os
#Mac
#os.chdir('/Users/yaircardenasavila/Desktop/Github_repo')
#Windows
os.chdir(r'C:\Users\armando_cardenas\Desktop\Github_repo\HH_Index')

In [6]:
cols_2026 = ['Clave Unidad Presupuestal', 'Unidad Presupuestal', 'Clave Unidad Responsable', 'Unidad Responsable', 'Clave Programa Presupuestario', 'Programa Presupuestario', 
        'Clave Componente', 'Componente', 'Clave Destino del Gasto Específico', 'Destino del Gasto Específico',
        'Clave Fuente de Financiamiento', 'Fuente de Financiamiento', 'Techo Comunicado', 'Monto Determinado', 'Adicional']
base_2026 = pd.read_excel('HH_Index\HH_Index\Fases de anteproyecto 2026 actualización de componentes transversalidad (1).xlsx', skiprows=1, usecols=cols_2026,
                          dtype={'Clave Unidad Presupuestal':str,
                                'Unidad Presupuestal':str,
                                'Clave Unidad Responsable':str,
                                'Unidad Responsable':str,
                                'Clave Programa Presupuestario':str,
                                'Programa Presupuestario':str,
                                'Clave Componente':str,
                                'Componente':str,
                                'Clave Destino del Gasto Específico':str,
                                'Destino del Gasto Específico':str,
                                'Clave Fuente de Financiamiento':str,
                                'Fuente de Financiamiento':str,
                                'Techo Comunicado':float,
                                'Monto Determinado':float,
                                'Adicional':float})

base_2026 = base_2026.rename(columns={'Concepto Capitulo 2':' NOMBRE Capitulo 2',
                              'Capitulo 2': 'CLAVE Capitulo 2'})
base_2026['llave'] = base_2026['Clave Unidad Presupuestal'] + '_' + base_2026['Clave Unidad Responsable']

<>:4: SyntaxWarning: invalid escape sequence '\H'
<>:4: SyntaxWarning: invalid escape sequence '\H'
C:\Users\armando_cardenas\AppData\Local\Temp\ipykernel_8860\4221289730.py:4: SyntaxWarning: invalid escape sequence '\H'
  base_2026 = pd.read_excel('HH_Index\HH_Index\Fases de anteproyecto 2026 actualización de componentes transversalidad (1).xlsx', skiprows=1, usecols=cols_2026,


#Análisis de cada PP con respecto a su UR

In [23]:
PP_UR = base_2026.groupby(by=['llave', 'Clave Programa Presupuestario']).agg({'Monto Determinado':'sum', 'Adicional':'sum'})
PP_UR['prop'] = (
    PP_UR['Monto Determinado'] /
    PP_UR.groupby(level=['llave'])['Monto Determinado'].transform('sum') 
)
PP_UR

Monto Determinado    Adicional      prop
llave  Clave Programa Presupuestario                                          
01_000 A01                                    7790003.0          0.0  1.000000
02_000 A02                                  528767891.0  254071771.0  0.983614
       SG1                                    8808588.0    1150000.0  0.016386
02_117 SPC                                  321151271.0   13412000.0  1.000000
02_172 SJ7                                  100000000.0          0.0  1.000000
...                                                 ...          ...       ...
52_000 SJ9                                   37939460.0    2300540.0  1.000000
53_000 S53                                  704371249.0          0.0  1.000000
54_000 S5V                                  197963000.0  664000000.0  1.000000
57_000 SRT                                   75553197.0   42627081.0  1.000000
58_000 A58                                  150000000.0  100000000.0  1.000000

[230 rows x 3 columns]

In [30]:
#Indice HH
PP_UR_HHI_N = (
    PP_UR.groupby(level="llave")["prop"]
    .apply(lambda s: ((s**2).sum() - 1/len(s)) / (1 - 1/len(s))).rename('HHI')
)
PP_UR_HHI_N

C:\Users\armando_cardenas\AppData\Local\Temp\ipykernel_8860\3757382724.py:4: RuntimeWarning: invalid value encountered in scalar divide
  .apply(lambda s: ((s**2).sum() - 1/len(s)) / (1 - 1/len(s))).rename('HHI')


llave
01_000         NaN
02_000    0.935531
02_117         NaN
02_172         NaN
02_173         NaN
            ...   
52_000         NaN
53_000         NaN
54_000         NaN
57_000         NaN
58_000         NaN
Name: HHI, Length: 135, dtype: float64

Análisis de cada comp con respecro a su PP

In [15]:
COMP_UR = base_2026.groupby(by=['llave', 'Clave Programa Presupuestario', 'Clave Componente']).agg({'Monto Determinado':'sum'})
COMP_UR['prop'] = (
    COMP_UR['Monto Determinado'] /
    COMP_UR.groupby(level=['llave', 'Clave Programa Presupuestario'])['Monto Determinado'].transform('sum')
)
COMP_UR

Monto Determinado  \
llave  Clave Programa Presupuestario Clave Componente                      
01_000 A01                           01                        7500052.0   
                                     02                         289951.0   
02_000 A02                           04                         435155.0   
                                     05                        6582270.0   
                                     08                         108330.0   
...                                                                  ...   
57_000 SRT                           A1                        4567144.0   
                                     A2                       51535935.0   
                                     A3                         600050.0   
                                     A4                       18850068.0   
58_000 A58                           C1                      150000000.0   

                                                           prop  
llave  Clave Programa Presupuestario Clave Componente            
01_000 A01                           01                0.962779  
                                     02                0.037221  
02_000 A02                           04                0.000823  
                                     05                0.012448  
                                     08                0.000205  
...                                                         ...  
57_000 SRT                           A1                0.060449  
                                     A2                0.682115  
                                     A3                0.007942  
                                     A4                0.249494  
58_000 A58                           C1                1.000000  

[1049 rows x 2 columns]

In [29]:
COMO_UR_HHI_N = (
    COMP_UR.groupby(level=['llave', 'Clave Programa Presupuestario'])["prop"]
    .apply(lambda s: ((s**2).sum() - 1/len(s)) / (1 - 1/len(s))).rename('HHI')
)
COMO_UR_HHI_N

C:\Users\armando_cardenas\AppData\Local\Temp\ipykernel_8860\1907271219.py:3: RuntimeWarning: invalid value encountered in scalar divide
  .apply(lambda s: ((s**2).sum() - 1/len(s)) / (1 - 1/len(s))).rename('HHI')


llave   Clave Programa Presupuestario
01_000  A01                              0.856658
02_000  A02                              0.682402
        SG1                              0.339741
02_117  SPC                              0.445289
02_172  SJ7                              0.369771
                                           ...   
52_000  SJ9                              0.068930
53_000  S53                              0.244733
54_000  S5V                              0.120140
57_000  SRT                              0.374993
58_000  A58                                   NaN
Name: HHI, Length: 230, dtype: float64

In [ ]:

base_2026.head()

,Clave Unidad Presupuestal,Unidad Presupuestal,Clave Unidad Responsable,Unidad Responsable,Clave Programa Presupuestario,Programa Presupuestario,Clave Componente,Componente,Clave Destino del Gasto Específico,Destino del Gasto Específico,Clave Fuente de Financiamiento,Fuente de Financiamiento,Techo Comunicado,Monto Determinado,Adicional,llave
0,01,Despacho del Gobernador,000,Despacho del Gobernador,A01,Atención Integral del Despacho del C. Gobernador,01,Gastos básicos para la operación,00,(Sin destino),15,Recursos Federales,4127904.0,4127904.0,0.0,01_000
1,01,Despacho del Gobernador,000,Despacho del Gobernador,A01,Atención Integral del Despacho del C. Gobernador,01,Gastos básicos para la operación,00,(Sin destino),15,Recursos Federales,26933.0,26933.0,0.0,01_000
2,01,Despacho del Gobernador,000,Despacho del Gobernador,A01,Atención Integral del Despacho del C. Gobernador,02,Atención Integral del Despacho del C. Gobernador,00,(Sin destino),15,Recursos Federales,57337.0,57337.0,0.0,01_000
3,01,Despacho del Gobernador,000,Despacho del Gobernador,A01,Atención Integral del Despacho del C. Gobernador,01,Gastos básicos para la operación,00,(Sin destino),15,Recursos Federales,573326.0,573326.0,0.0,01_000
4,01,Despacho del Gobernador,000,Despacho del Gobernador,A01,Atención Integral del Despacho del C. Gobernador,01,Gastos básicos para la operación,00,(Sin destino),15,Recursos Federales,161883.0,161883.0,0.0,01_000
